In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

tqdm.pandas()

In [2]:
page = requests.get("https://beallslist.net/standalone-journals/")
soup = BeautifulSoup(page.content, "html.parser")

In [3]:
for ul in soup.find_all("ul", class_="main-menu"):
    ul.decompose()
    # print(ul.prettify())

In [4]:
for ul in soup.find_all("ul", class_="sub-menu"):
    ul.decompose()
    # print(ul.prettify())

In [5]:
links = soup.select("ul > li > a")

names = []
urls = []

for link in links:
    names.append(link.contents[0])
    urls.append(link.get("href"))

df = pd.DataFrame({"source": names, "url": urls})

In [6]:
df

,source,url
0,Academic Exchange Quarterly,http://www.rapidintellect.com/AEQweb/
1,Academic Research Reviews,http://academicreviews.us/Default.aspx
2,Academy of Contemporary Research Journal,http://aocrj.org/
3,ACME Intellects,http://www.acmeintellects.org/
4,Acta de Gerencia Ciencia,http://www.cagena.com/
...,...,...
1540,Technology Reports of Kansai University,http://kansaiuniversityreports.com/
1541,TEST: Engineering & Management,http://www.testmagzine.biz/
1542,"The Turkish Nephrology, Dialysis and Transplan...",http://www.tndt.org/
1543,World Journal of Environmental Biosciences,http://www.environmentaljournals.org/


In [7]:
def url_ok(url):
    try:
        r = requests.head(url, timeout=30)
        return r.status_code
    except:
        return None

# def url_ok2(url):
#     try:
#         return urllib.request.urlopen(url).getcode()
#     except:
#         return None

In [8]:
df["status"] = df.progress_apply(lambda row: url_ok(row["url"]), axis=1)

100%|██████████| 1545/1545 [26:32<00:00,  1.03s/it]  


In [9]:
df["status"].value_counts(dropna=False, normalize=True)

200.0    0.377346
301.0    0.245955
406.0    0.116505
NaN      0.109385
302.0    0.082201
403.0    0.029773
404.0    0.022654
500.0    0.007120
400.0    0.003883
412.0    0.001942
401.0    0.001294
503.0    0.000647
303.0    0.000647
308.0    0.000647
Name: status, dtype: float64

In [10]:
df["date"] = datetime.today().strftime('%B %d, %Y')

In [11]:
df.to_csv("../data/df_weekly_standalone.csv", encoding='utf-8', index=False)